In [2]:
%%pyspark
df = spark.read \
.option('inferSchema', 'true') \
.json('abfss://raw042772@vksa042772.dfs.core.windows.net/wwi-02/online-user-profiles-02/', multiLine=True)
display(df.limit(10))

StatementMeta(venkysparkpool, 3, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, 04079d95-3f50-4e8c-84f1-5743f2b27bc4)

In [3]:
df.printSchema()

StatementMeta(venkysparkpool, 3, 3, Finished, Available)

root
 |-- topProductPurchases: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- itemsPurchasedLast12Months: long (nullable = true)
 |    |    |-- productId: long (nullable = true)
 |-- visitorId: long (nullable = true)

In [4]:
df.createOrReplaceTempView('user_profiles')

StatementMeta(venkysparkpool, 3, 4, Finished, Available)

In [6]:
%%sql
select * from user_profiles limit 10

StatementMeta(venkysparkpool, 3, 6, Finished, Available)

<Spark SQL result set with 10 rows and 2 fields>

In [8]:
from pyspark.sql.functions import udf, explode
flat = df.select("visitorId", explode("topProductPurchases").alias("topProductPurchases_flat"))
flat.show()

StatementMeta(venkysparkpool, 3, 8, Finished, Available)

+---------+------------------------+
|visitorId|topProductPurchases_flat|
+---------+------------------------+
|   117000|              {13, 3623}|
|   117000|               {5, 2321}|
|   117001|               {93, 713}|
|   117001|              {19, 2144}|
|   117001|              {30, 1094}|
|   117001|              {82, 3223}|
|   117001|              {42, 3328}|
|   117001|              {62, 2926}|
|   117001|              {63, 2651}|
|   117001|               {39, 341}|
|   117001|              {85, 4841}|
|   117001|              {67, 4289}|
|   117001|              {42, 1264}|
|   117001|              {43, 3608}|
|   117001|               {14, 504}|
|   117001|              {97, 2649}|
|   117001|              {44, 2873}|
|   117001|               {7, 4491}|
|   117001|              {70, 3623}|
|   117001|               {4, 4035}|
+---------+------------------------+
only showing top 20 rows

In [10]:
topPurchases = flat.select('visitorId', 'topProductPurchases_flat.productId', 'topProductPurchases_flat.itemsPurchasedLast12Months').orderBy('visitorId')
topPurchases.show()

StatementMeta(venkysparkpool, 3, 10, Finished, Available)

+---------+---------+--------------------------+
|visitorId|productId|itemsPurchasedLast12Months|
+---------+---------+--------------------------+
|    80000|     2859|                        54|
|    80000|     4074|                        48|
|    80000|     1362|                        18|
|    80000|      102|                        42|
|    80000|     3794|                        90|
|    80000|     2481|                        63|
|    80000|     3538|                        65|
|    80000|     4024|                        35|
|    80000|     4206|                        21|
|    80000|     3122|                        33|
|    80000|       93|                        86|
|    80000|     1330|                        78|
|    80000|     2488|                        31|
|    80000|      290|                        83|
|    80000|      291|                        49|
|    80000|     2069|                        93|
|    80000|     3270|                         5|
|    80000|     4745

In [20]:
sortedTopPurchases = topPurchases.orderBy(topPurchases["itemsPurchasedLast12Months"].desc())
display(sortedTopPurchases)

StatementMeta(venkysparkpool, 3, 20, Finished, Available)

SynapseWidget(Synapse.DataFrame, f81a1d86-809b-47f0-8b75-c125eced2efb)

In [22]:
from pyspark.sql.functions import count
groupedTopPurchases = sortedTopPurchases.select("visitorId").groupBy("visitorId").agg(count("*").alias("total")).orderBy("visitorId")
display(groupedTopPurchases)

StatementMeta(venkysparkpool, 3, 22, Finished, Available)

SynapseWidget(Synapse.DataFrame, b759fcfa-a59f-490c-9d5f-98056bf06315)

In [24]:
from pyspark.sql.functions import sum
groupedTopPurchases1 = sortedTopPurchases.select("visitorId", "itemsPurchasedLast12Months").groupBy("visitorId").agg(sum("itemsPurchasedLast12Months").alias("totalItemsPurchased")).orderBy("visitorId")
display(groupedTopPurchases1)

StatementMeta(venkysparkpool, 3, 24, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5b26a4c2-99e7-4efd-9d20-43c2007337e1)

In [25]:
# Create a temporary view for top purchases so we can load from Scala
topPurchases.createOrReplaceTempView("top_purchases")

StatementMeta(venkysparkpool, 3, 25, Finished, Available)

In [27]:
%%spark
// Make sure the name of the dedcated SQL pool (SQLPool01 below) matches the name of your SQL pool.
val df = spark.sqlContext.sql("select * from top_purchases")
df.write.synapsesql("venkysqlpool.wwi.TopPurchases", Constants.INTERNAL)

StatementMeta(venkysparkpool, 3, 28, Finished, Available)

df: org.apache.spark.sql.DataFrame = [visitorId: bigint, productId: bigint ... 1 more field]


In [28]:
dfsales = spark.read.load('abfss://raw042772@vksa042772.dfs.core.windows.net/wwi-02/sale-small/Year=2019/Quarter=Q4/Month=12/*/*.parquet', format='parquet')
display(dfsales.limit(10))

StatementMeta(venkysparkpool, 3, 29, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3994ba0f-b552-4b58-816b-3cf4e2494961)

In [29]:
%%spark
// Make sure the name of the SQL pool (SQLPool01 below) matches the name of your SQL pool.
val df2 = spark.read.synapsesql("venkysqlpool.wwi.TopPurchases")
df2.createTempView("top_purchases_sql")

df2.head(10)

StatementMeta(venkysparkpool, 3, 30, Finished, Available)

df2: org.apache.spark.sql.DataFrame = [visitorId: bigint, productId: bigint ... 1 more field]
res9: Array[org.apache.spark.sql.Row] = Array([115084,4401,68], [115084,2836,32], [115084,530,26], [115084,394,89], [115084,2014,34], [115084,529,95], [115084,3198,20], [115084,3214,33], [115084,3580,67], [115084,3464,62])


In [30]:
dfTopPurchasesFromSql = sqlContext.table("top_purchases_sql")

display(dfTopPurchasesFromSql.limit(10))

StatementMeta(venkysparkpool, 3, 31, Finished, Available)

SynapseWidget(Synapse.DataFrame, e32b9f2a-b162-4bae-99a1-5571166c6a3b)

In [31]:
inner_join = dfsales.join(dfTopPurchasesFromSql,
    (dfsales.CustomerId == dfTopPurchasesFromSql.visitorId) & (dfsales.ProductId == dfTopPurchasesFromSql.productId))

inner_join_agg = (inner_join.select("CustomerId","TotalAmount","Quantity","itemsPurchasedLast12Months","top_purchases_sql.productId")
    .groupBy(["CustomerId","top_purchases_sql.productId"])
    .agg(
        sum("TotalAmount").alias("TotalAmountDecember"),
        sum("Quantity").alias("TotalQuantityDecember"),
        sum("itemsPurchasedLast12Months").alias("TotalItemsPurchasedLast12Months"))
    .orderBy("CustomerId") )

display(inner_join_agg.limit(100))

StatementMeta(venkysparkpool, 3, 32, Finished, Available)

SynapseWidget(Synapse.DataFrame, aae634ac-4e05-4e41-afbf-1f8b0c8b3829)